In [32]:
import pandas as pd

raw_path = r"f:/@courseworks/PhysicsExperiments/ThermalResistor/processed_data.csv"

# Read all rows as lists, handling inconsistent columns
with open(raw_path, encoding='utf-8') as f:
    lines = [line.strip() for line in f if line.strip()]

rows = [line.split(',') for line in lines]

# Find the maximum number of columns
max_cols = max(len(row) for row in rows)

# Pad rows so all have the same length
for row in rows:
    row += [''] * (max_cols - len(row))

# Extract variable names and their data
data_dict = {}
for row in rows:
    key = row[0]
    values = row[1:]
    data_dict[key] = values

# Convert to DataFrame (transpose so each variable is a column)
df = pd.DataFrame(data_dict)
df

# Plotting 1/T vs lnR_T
This cell demonstrates how to plot the variation of $1/T$ with $\ln R_T$ using matplotlib.

In [61]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

# Set font to support Chinese characters
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'Microsoft YaHei', 'STSong', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False

# Extract and clean 1/T and lnR_T columns (skip empty/invalid values)
x = df['1/T (10^-2 K^-1)']
y = df['ln R_T']

# Convert to numeric, coerce errors to NaN, then drop NaNs
x = pd.to_numeric(x, errors='coerce')
y = pd.to_numeric(y, errors='coerce')
mask = (~x.isna()) & (~y.isna())
x = x[mask]
y = y[mask]

# Define linear function model
def linear_func(x, k, b):
    return k * x + b

# Fit data using scipy's curve_fit
popt, pcov = curve_fit(linear_func, x, y)
k, b = popt
perr = np.sqrt(np.diag(pcov))  # Standard errors of the parameters
k_err, b_err = perr

# Generate fitted line
x_fit = np.linspace(x.min(), x.max(), 100)
y_fit = linear_func(x_fit, k, b)

# 计算横纵轴范围并扩展10%
x_min, x_max = x.min(), x.max()
y_min, y_max = y.min(), y.max()
x_pad = (x_max - x_min) * 0.1
xlim = (x_min - x_pad, x_max + x_pad)
y_pad = (y_max - y_min) * 0.1
ylim = (y_min - y_pad, y_max + y_pad)

plt.figure(figsize=(7,5))
plt.scatter(x, y, color='b', label=fr'$\mathrm{{Data}}$')
plt.plot(x_fit, y_fit, 'r-', 
         label=fr'$\ln R_T = ({k:.2f}\pm{k_err:.2f})x {b:+.2f}\pm{b_err:.2f}$')
plt.xlabel(r'$1/T\ (10^{-2}\ \mathrm{K}^{-1})$')
plt.ylabel(r'$\ln R_T$')
plt.title(r'$1/T$ 与 $\ln R_T$ 关系图')
plt.xlim(xlim)
plt.ylim(ylim)
plt.grid(True)
plt.legend()
plt.savefig('output/plot1.png', dpi=300, bbox_inches='tight')
plt.show()